# Analysing Telco Company's Customer Churn Rate

- The goal of this project is to analyse the customer base of the given telco company and to predict the probability of churn.

- The dataset was created by IBM and contains information about services provided to 7043 customers in California in Q3. A more detailed description is available on [IBM's website](https://community.ibm.com/community/user/blogs/steven-macko/2019/07/11/telco-customer-churn-1113).


The original dataset is provided by IBM in XLSX format. For the purpose of a more realistic analysis environment, the data has been moved into a PostgreSQL database inside a Docker container. The data entries were uploaded using pdAdmin. The schema is described in `schema.sql`.

### Connecting to the database

In [1]:
%load_ext sql

In [2]:
user = 'postgres'
password = 'password'
host = '127.0.0.1'
port = 5431
database = 'telco-db'

connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

%sql $connection_string

%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
%%sql
SELECT table_name AS "Table Name"
FROM information_schema.tables
WHERE table_schema NOT IN ('pg_catalog', 'information_schema')

Table Name
local_population
demographics
location
services
status


## Database overview

### `location` table

In [ ]:
%%sql
SELECT *
FROM location
LIMIT 5

location_id,customer_id,count,country,state,city,zip_code,lat_long,latitude,longitude
OXCZEW7397,8779-QRDMV,1,United States,California,Los Angeles,90022,"34.02381, -118.156582",34.02381,-118.156582
FCCECI8494,7495-OOKFY,1,United States,California,Los Angeles,90063,"34.044271, -118.185237",34.044271,-118.185237
HEHUQY7254,1658-BYGOY,1,United States,California,Los Angeles,90065,"34.108833, -118.229715",34.108833,-118.229715
WIUHRF2613,4598-XLKNJ,1,United States,California,Inglewood,90303,"33.936291, -118.332639",33.936291,-118.332639
CFEZBF4415,4846-WHAFZ,1,United States,California,Whittier,90602,"33.972119, -118.020188",33.972119,-118.020188


This table contains location details of customers that are all based in California. The primary key of this table is composite and made up of `location_id` and `customer_id`.

### `services` table

In [5]:
%%sql
SELECT *
FROM services
LIMIT 5

service_id,customer_id,count,quarter,referred_a_friend,n_referred,months_tenure,offer,phone_service,avg_monthly_long_dist_charge,multiple_lines,internet_service,internet_type,avg_monthly_gb_down,online_security,online_backup,device_protection,premium_tech_support,streaming_tv,streaming_movies,streaming_music,unlimited_data,contract_type,paperless_billing,payment_method,monthly_charge,total_charge,total_refunds,total_extra_data_charges,total_long_dist_charges,total_revenue
IJKDQVSWH3522,8779-QRDMV,1,Q3,No,0,1,None,No,0.0,No,Yes,DSL,8,No,No,Yes,No,No,Yes,No,No,Month-to-Month,Yes,Bank Withdrawal,39.65,39.65,0.0,20.0,0.0,59.65
BFKMZJAIE2285,7495-OOKFY,1,Q3,Yes,1,8,Offer E,Yes,48.85,Yes,Yes,Fiber Optic,17,No,Yes,No,No,No,No,No,Yes,Month-to-Month,Yes,Credit Card,80.65,633.3,0.0,0.0,390.8,1024.1
EIMVJQBMT7187,1658-BYGOY,1,Q3,No,0,18,Offer D,Yes,11.33,Yes,Yes,Fiber Optic,52,No,No,No,No,Yes,Yes,Yes,Yes,Month-to-Month,Yes,Bank Withdrawal,95.45,1752.55,45.61,0.0,203.94,1910.88
EROZQXDUU4979,4598-XLKNJ,1,Q3,Yes,1,25,Offer C,Yes,19.76,No,Yes,Fiber Optic,12,No,Yes,Yes,No,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.5,2514.5,13.43,0.0,494.0,2995.07
GEEYSJUHY6991,4846-WHAFZ,1,Q3,Yes,1,37,Offer C,Yes,6.33,Yes,Yes,Fiber Optic,14,No,No,No,No,No,No,No,Yes,Month-to-Month,Yes,Bank Withdrawal,76.5,2868.15,0.0,0.0,234.21,3102.36


In [6]:
%%sql
SELECT column_name AS "Column Name"
FROM information_schema.columns
WHERE table_catalog = 'telco-db' AND table_name = 'services'
ORDER BY ordinal_position

Column Name
service_id
customer_id
count
quarter
referred_a_friend
n_referred
months_tenure
offer
phone_service
avg_monthly_long_dist_charge


This table contains client-specific information about the service provided and the main information that will be useful when modelling churn.

### `status` table

In [7]:
%%sql
SELECT *
FROM status
LIMIT 5

status_id,customer_id,count,quarter,satisfaction_score,customer_status,churn_label,churn_value,church_score,cltv,churn_category,churn_reason
SWSORB1252,8779-QRDMV,1,Q3,3,Churned,Yes,True,91,5433,Competitor,Competitor offered more data
SNAEQA8572,7495-OOKFY,1,Q3,3,Churned,Yes,True,69,5302,Competitor,Competitor made better offer
LMBQNN3714,1658-BYGOY,1,Q3,2,Churned,Yes,True,81,3179,Competitor,Competitor made better offer
VRZYZI9978,4598-XLKNJ,1,Q3,2,Churned,Yes,True,88,5337,Dissatisfaction,Limited range of services
FDNAKX1688,4846-WHAFZ,1,Q3,2,Churned,Yes,True,67,2793,Price,Extra data charges


This table shows information regarding the currect customer's satisfaction and whether they have or have not churned. Information about the cause of churn is also provided along with customer lifetime value which can also be modelled.

### `local_population` table

In [8]:
%%sql
SELECT *
FROM local_population
LIMIT 5

id,zip_code,population
1,90001,54492
2,90002,44586
3,90003,58198
4,90004,67852
5,90005,43019


This table shows the local population in a given zip code. This can be used to discert more urban areas from more rural ones.

### `demographics` table

In [9]:
%%sql
SELECT *
FROM demographics
LIMIT 5

customer_id,count,gender,age,under_30,is_senior,is_married,has_dependents,n_dependents
8779-QRDMV,1,Male,78,No,Yes,No,No,0
7495-OOKFY,1,Female,74,No,Yes,Yes,Yes,1
1658-BYGOY,1,Male,71,No,Yes,No,Yes,3
4598-XLKNJ,1,Female,78,No,Yes,Yes,Yes,1
4846-WHAFZ,1,Female,80,No,Yes,Yes,Yes,1


This table describes the person's demographic details such as age, gender and family status.

## Exploring the data

One way to analyse this data would be merging all tables and performing analysys on Pandas's DataFrame but let's instead perform basic analysis through SQL itself.

#### How long have our customers been with the company?

In [10]:
%%sql
SELECT MIN(months_tenure),
PERCENTILE_CONT(0.25) WITHIN GROUP (
   ORDER BY months_tenure) AS q25,
PERCENTILE_CONT(0.5) WITHIN GROUP (
   ORDER BY months_tenure) AS median,
AVG(months_tenure),
PERCENTILE_CONT(0.75) WITHIN GROUP (
   ORDER BY months_tenure) AS q75,
MAX(months_tenure)
FROM status LEFT JOIN services ON status.customer_id = services.customer_id
GROUP BY churn_value

min,q25,median,avg,q75,max
1,15.0,38.0,37.5912253575570158,61.0,72
1,2.0,10.0,17.9791332263242376,29.0,72


The data shows us that churned customers churn earlier than those that have stayed - Q75 for churned customers is well below the median for the non-churned ones. We have clear separation. That is positive news as it means that our loyal customers are choosing to stay.